In [1]:
from laptrack import LapTrack
from laptrack import data_conversion
from laptrack import datasets
import geff
import tempfile
from skimage.measure import regionprops_table
import pandas as pd
from pathlib import Path
import zarr
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
tmp_path = Path(tempfile.gettempdir())
print(f"Temporary files will be stored in: {tmp_path}")

Temporary files will be stored in: /var/folders/pz/_dlzclyn4mjgvngf1dpfg_880000gn/T


# Example of reading / saving data from / to GEFF

This example illustrates how we can save the tracked data into a [GEFF (Graph Exchange File Format)](https://github.com/live-image-tracking-tools/geff).

The GEFF file can be visualized in `napari` by [napari-geff](https://github.com/live-image-tracking-tools/napari-geff).

## Preparing the data

### Calculate region properties 

In [2]:
image, labels = datasets.cell_segmentation()
regionprops = []
for frame, label in enumerate(labels):
    df = pd.DataFrame(regionprops_table(label, properties=["label", "centroid"]))
    df["frame"] = frame
    regionprops.append(df)
regionprops_df = pd.concat(regionprops).rename(
    columns={"centroid-0": "y", "centroid-1": "x", "label": "seg_id"}
)
coordinate_cols = ["y", "x"]
frame_col = "frame"
regionprops_df.head()

,seg_id,y,x,frame
0,1,137.033613,96.336134,0
1,2,145.068966,178.988506,0
2,3,123.091743,203.266055,0
3,4,62.716216,141.290541,0
4,5,32.976744,124.348837,0


### Track with LapTrack

In [3]:
lt = LapTrack(
    metric="sqeuclidean",
    cutoff=15**2,
    splitting_metric="sqeuclidean",
    merging_metric="sqeuclidean",
    splitting_cutoff=15**2,
    merging_cutoff=15**2,
)

In [4]:
regionprops_df.head()

,seg_id,y,x,frame
0,1,137.033613,96.336134,0
1,2,145.068966,178.988506,0
2,3,123.091743,203.266055,0
3,4,62.716216,141.290541,0
4,5,32.976744,124.348837,0


In [5]:
track_df, split_df, merge_df = lt.predict_dataframe(
    regionprops_df,
    coordinate_cols=coordinate_cols,
    frame_col=frame_col,
)

track_df.head()

,seg_id,y,x,frame,tree_id,track_id
0,1,137.033613,96.336134,0,0,0
1,2,145.068966,178.988506,0,1,1
2,3,123.091743,203.266055,0,2,2
3,4,62.716216,141.290541,0,3,3
4,5,32.976744,124.348837,0,4,4


## Save to GEFF

In [6]:
tree_geff = data_conversion.dataframes_to_geff_networkx(
    track_df,
    split_df,
    merge_df,
    frame_col=frame_col,
)

In [7]:
tree_geff.nodes[0]

{'frame': 0,
 'seg_id': np.float64(1.0),
 'y': np.float64(137.03361344537817),
 'x': np.float64(96.33613445378151),
 'tree_id': np.float64(0.0),
 'track_id': np.float64(0.0)}

In [8]:
zarr_path = Path(tmp_path) / "cell_segmentation.zarr"
ds = zarr.open(zarr_path, mode="w")
ds.create_dataset("image", data=image)
ds.create_dataset("labels", data=labels)

meta = geff.GeffMetadata(
    related_objects=[
        {"type": "image", "path": "../image"},
        {"type": "labels", "path": "../labels", "label_prop": "seg_id"},
    ],
    track_node_props={"lineage": "tree_id", "tracklet": "track_id"},
    directed=True,
)

geff.write_nx(
    tree_geff,
    zarr_path / "track.geff",
    metadata=meta,
    axis_names=[frame_col] + coordinate_cols,
    axis_types=["time", "space", "space"],
    axis_units=["frame", "px", "px"],
)
geff.validate(zarr_path / "track.geff")
!ls -lh {zarr_path}/track.geff/nodes/props/

total 0
drwxr-xr-x 4 fukai staff 128 Jul 25 11:37 frame
drwxr-xr-x 4 fukai staff 128 Jul 25 11:37 seg_id
drwxr-xr-x 4 fukai staff 128 Jul 25 11:37 track_id
drwxr-xr-x 4 fukai staff 128 Jul 25 11:37 tree_id
drwxr-xr-x 4 fukai staff 128 Jul 25 11:37 x
drwxr-xr-x 4 fukai staff 128 Jul 25 11:37 y
